# Libraries

In [1]:
import sys
import os
import scanpy as sc
import pandas as pd
import numpy as np

# Configurations

In [2]:
meta = pd.read_csv('meta_all_gene.csv')
meta = meta[meta['species'] == 'human']

path = "../data"

In [3]:
meta

,slide,species,tissue,pmid,title,abstract,keywords,involve_cancer,tech,spot_num,gene_num
0,GSE144239_GSM4284316,human,skin,"32579974,38037084",Title 1: Multimodal Analysis of Composition an...,Abstract 1: To define the cellular composition...,Keywords 1: CRISPR screen; MIBI; intra-tumoral...,True,ST,666,17138
1,GSE144239_GSM4284317,human,skin,"32579974,38037084",Title 1: Multimodal Analysis of Composition an...,Abstract 1: To define the cellular composition...,Keywords 1: CRISPR screen; MIBI; intra-tumoral...,True,ST,646,17344
2,GSE144239_GSM4284318,human,skin,"32579974,38037084",Title 1: Multimodal Analysis of Composition an...,Abstract 1: To define the cellular composition...,Keywords 1: CRISPR screen; MIBI; intra-tumoral...,True,ST,638,17883
3,GSE144239_GSM4284319,human,skin,"32579974,38037084",Title 1: Multimodal Analysis of Composition an...,Abstract 1: To define the cellular composition...,Keywords 1: CRISPR screen; MIBI; intra-tumoral...,True,ST,590,16959
4,GSE144239_GSM4284320,human,skin,"32579974,38037084",Title 1: Multimodal Analysis of Composition an...,Abstract 1: To define the cellular composition...,Keywords 1: CRISPR screen; MIBI; intra-tumoral...,True,ST,521,17689
...,...,...,...,...,...,...,...,...,...,...,...
1114,Human_Prostate_Erickson_08102022_Visium_Patien...,human,prostate,35948708,Spatially resolved clonal copy number alterati...,Defining the transition from benign to maligna...,NaN,True,Visium,3190,33538
1115,Human_Prostate_Erickson_08102022_Visium_Patien...,human,prostate,35948708,Spatially resolved clonal copy number alterati...,Defining the transition from benign to maligna...,NaN,True,Visium,3554,33538
1116,Human_Prostate_Erickson_08102022_Visium_Patien...,human,prostate,35948708,Spatially resolved clonal copy number alterati...,Defining the transition from benign to maligna...,NaN,True,Visium,2736,33538
1145,Human_Colon_10X_03252024_VisiumHD,human,colon,NaN,NaN,NaN,NaN,False,VisiumHD,545913,18085


In [4]:
def find_hvg(slide):
    try:
        gene_exp_slide = pd.read_csv(f'{path}/Visium/gene_exp/{slide}_count.csv',sep=',',index_col=0)
        adata = sc.AnnData(gene_exp_slide)
        adata.var_names_make_unique()
        
        sc.pp.filter_cells(adata, min_genes=1)
        sc.experimental.pp.highly_variable_genes(adata, n_top_genes=128)
        
        # sort genes by highly_variable
        adata.var_names_make_unique()
        hvg_list = adata.var['highly_variable_rank']
        hvg_list = hvg_list.sort_values()
        hvg_list = hvg_list.dropna()
        
        adata_hvg = adata[:, hvg_list.index]
        sc.pp.normalize_total(adata_hvg)
        sc.pp.log1p(adata_hvg)
        hvg = adata_hvg.X
        hvg = pd.DataFrame(hvg)
        hvg.index = adata_hvg.obs.index
        hvg.to_csv(f'../data/HVG/{slide}_count_hvg.csv',sep=',')
        print(f"FOUND HVG: {path}/Visium/gene_exp/{slide}_count.csv")
    except:
        pass

# Find HVGs

In [5]:
tissues= meta['tissue'].unique()

for tissue in tissues:
    print(f"===Processing Tissue: {tissue}===")
    # filter the meta file to get only one tissue at a time
    filtered_meta = meta[(meta['tissue'] == tissue)]
    
    # =========================
    def save_slides():
        try:
            # Define file path
            slide_dir = os.path.join("../data/slides")
            os.makedirs(slide_dir, exist_ok=True)
    
            filtered_meta['slide'].to_csv(f"{slide_dir}/{tissue}_slide.csv", index=False, header=False)
            print(f"Success Slide file created for tissue: {tissue}")
        except:
            print(f"Error Creating slide for tissue: {tissue}")

    save_slides()
    # =========================
    def find_hvgs():
        try:
            # Process each row efficiently
            os.makedirs("../data/HVG", exist_ok=True)
    
            slide_file = f'../data/slides/{tissue}_slide.csv'
            with open(slide_file, 'r') as f:
                    slides = f.read().splitlines()
            
            for slide in slides:
                find_hvg(slide)
            print(f"Success HVG files created for tissue: {tissue}")
        except:
            print(f"Error creating HVG files for tissue: {tissue}")
            
    find_hvgs()
    # =========================

    def combine_hvg():
        try:
            path = '../data'
            combined_hvg_dir='../data/combined_hvg/'
            output_file = f'{combined_hvg_dir}{tissue}_count_hvg.csv'
            slide_file = f'../data/slides/{tissue}_slide.csv'
    
            os.makedirs(combined_hvg_dir, exist_ok=True)
    
            # Remove the output file if it exists
            if os.path.exists(output_file):
                os.remove(output_file)
            
            # Read slide names from file
            with open(slide_file, 'r') as f:
                slides = f.read().splitlines()
            
            # Process each slide file
            for slide in slides:
                input_file = os.path.join(path, 'HVG', f'{slide}_count_hvg.csv')
                if os.path.exists(input_file):
                    df = pd.read_csv(input_file, skiprows=1)  # Skip the first row (header)
                    df.to_csv(output_file, mode='a', index=False, header=False)
                # print(slide)
            print(f"Success HVG files combined for tissue: {tissue}")
        except:
            print(f"Error combining HVG files for tissue: {tissue}")

    combine_hvg()
    # =========================
    def find_overlap_genes():
        try:
            gene_name_overlap = []
            data = filtered_meta
            data.index = range(len(data.index))
            slides = pd.read_csv(f"../data/slides/{tissue}_slide.csv")
            slides = slides.iloc[:, 0].tolist()
            
            i = 0
            for slide in slides:
                
                try:
                    gene_exp_slide = pd.read_csv(f'../data/Visium/gene_exp/{slide}_count.csv',sep=',',nrows=1,index_col=0)
                    if i == 0:
                        gene_name_overlap = gene_exp_slide.columns
                    else:
                        gene_name_overlap = gene_name_overlap.intersection(gene_exp_slide.columns)
                    i += 1
                except:
                    pass
            
            if len(gene_name_overlap) != 0:
                save_dir = "../data/overlap-genes"
                os.makedirs(save_dir, exist_ok=True)
                pd.DataFrame(gene_name_overlap).to_csv(f'{save_dir}/{tissue}_gene.csv',index=False,header=False)
            print(f"Success Overlap genes found for tissue: {tissue}")
        except:
            print(f"Error finding overlap genes for tissue: {tissue}")
            
    find_overlap_genes()
    print(f"$ HVG FILES OVERLAPPED FOR {tissue}")
    # =========================
    def extract_overlap_gene():
        try:
            gene_list = tissue+'_gene'
            slide_file = f'../data/slides/{tissue}_slide.csv'
            with open(slide_file, 'r') as f:
                slides = f.read().splitlines()
            
            for slide in slides:
                try:
                    gene_exp_slide = pd.read_csv(f'../data/Visium/gene_exp/{slide}_count.csv',sep=',',index_col=0)
                    overlap_gene = pd.read_csv(f'../data/overlap-genes/{gene_list}.csv',header=None)
                    gene_exp_slide = gene_exp_slide.loc[:,overlap_gene[0]]
                    os.makedirs("../data/overlap-hvg", exist_ok=True)
                    gene_exp_slide.to_csv(f'../data/overlap-hvg/{slide}_{gene_list}.csv',sep=',')
                    print(f"DONE: ../data/Visium/gene_exp/{slide}_count.csv")
                except:
                    pass
            print(f"Success overlap genes extracted for tissue: {tissue}")
        except:
            print(f"Error extracting overlap genes for tissue: {tissue}")
            
    extract_overlap_gene()
    # =========================
    def overlap_tissue():
        try:
            # Define variables
            path = '../data/overlap-hvg/'
            output_file = f'../data/overlap-tissue/{tissue}_count_overlap.csv'
            slide_file = f'../data/slides/{tissue}_slide.csv'
    
            os.makedirs("../data/overlap-tissue", exist_ok=True)
            # Remove existing output file if it exists
            if os.path.exists(output_file):
                os.remove(output_file)
    
            # Read slide names
            with open(slide_file, 'r') as file:
                slides = file.read().splitlines()
    
            # Process slides
            missing_slides = []
            header_written = False
    
            for slide in slides:
                input_file = os.path.join(path, f"{slide}_{tissue}_gene.csv")
            
                if os.path.isfile(input_file):
                    df = pd.read_csv(input_file, skiprows=1)  # Skip the first row (header)
                    df.to_csv(output_file, mode='a', index=False, header=False)  # Append without header
                else:
                    missing_slides.append(slide)
            print(f"Success overlapped tissue: {tissue}")
        except:
            print(f"Error overlapping tissue: {tissue}")
            

    overlap_tissue()
    # =========================
    def overlap_hvg():
        try:
            # Here we take input of combined gene expression of one tissue type
            gene_exp_slide = pd.read_csv(f'../data/overlap-tissue/{tissue}_count_overlap.csv',sep=',',index_col=0,header=None)
            gene_exp_slide.columns = ['gene'+str(i) for i in range(gene_exp_slide.shape[1])]

            adata = sc.AnnData(gene_exp_slide)
            sc.pp.filter_cells(adata, min_genes=1)
            sc.pp.filter_genes(adata, min_cells=1)
            # Normalizing to median total counts
            sc.pp.normalize_total(adata)
            # Logarithmize the data
            sc.pp.log1p(adata)
            sc.pp.highly_variable_genes(adata, n_top_genes=100)
            # sort genes by highly_variable
            adata_hvg = adata[:, adata.var.highly_variable]
            hvg = adata_hvg.X
            hvg = pd.DataFrame(hvg)
            hvg.index = adata_hvg.obs.index

            os.makedirs("../data/all_genes", exist_ok=True)
            hvg.to_csv(f'../data/all_genes/{tissue}_count_overlap_hvg.csv',sep=',')
            print(f"Success overlapped hvg tissue: {tissue}")
        except:
            print(f"Error overlapping hvg tissue: {tissue}")

    overlap_hvg()
    # =========================
    print(f"PREPROCESSING COMPLETED FOR TISSUE: {tissue}")


===Processing Tissue: skin===
Success Slide file created for tissue: skin


/home/amro/projects/tf217/tf217/lib/python3.12/site-packages/scanpy/preprocessing/_normalization.py:208: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


FOUND HVG: ../data/Visium/gene_exp/GSE144239_GSM4565823_count.csv


/home/amro/projects/tf217/tf217/lib/python3.12/site-packages/scanpy/preprocessing/_normalization.py:208: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


FOUND HVG: ../data/Visium/gene_exp/GSE144239_GSM4565824_count.csv
Success HVG files created for tissue: skin
Success HVG files combined for tissue: skin
Success Overlap genes found for tissue: skin
$ HVG FILES OVERLAPPED FOR skin
DONE: ../data/Visium/gene_exp/GSE144239_GSM4565823_count.csv
DONE: ../data/Visium/gene_exp/GSE144239_GSM4565824_count.csv
Success overlap genes extracted for tissue: skin
Success overlapped tissue: skin
Success overlapped hvg tissue: skin
PREPROCESSING COMPLETED FOR TISSUE: skin
===Processing Tissue: breast===
Success Slide file created for tissue: breast
Success HVG files created for tissue: breast
Success HVG files combined for tissue: breast
Success Overlap genes found for tissue: breast
$ HVG FILES OVERLAPPED FOR breast
Success overlap genes extracted for tissue: breast
Success overlapped tissue: breast
Error overlapping hvg tissue: breast
PREPROCESSING COMPLETED FOR TISSUE: breast
===Processing Tissue: heart===
Success Slide file created for tissue: heart